In [9]:
from keras.models import load_model
from SiameseModel import SiameseModel
from SiameseLoader import SiameseLoader
import glob
import os
import numpy as np

In [2]:
ckpts = glob.glob("checkpoints/*.hdf5")
latest_ckpt = max(ckpts, key=os.path.getctime)
print("loading from checkpoint: ", latest_ckpt)
initial_epoch = int(latest_ckpt[latest_ckpt.find("-epoch-") + len("-epoch-"):latest_ckpt.rfind("-lr-")])
model = load_model(latest_ckpt)

loading from checkpoint:  checkpoints/flowchroma-epoch-00480-lr--train_loss-0.6302-val_loss-0.5263.hdf5


/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:1271: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [3]:
loader = SiameseLoader('./data')

loading snake breed: 152
loading snake breed: 159
loading snake breed: 80
loading snake breed: 187


/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


loading snake breed: 3
loading snake breed: 46
loading snake breed: 158
loading snake breed: 87
loading snake breed: 123
loading snake breed: 100
loading snake breed: 205
loading snake breed: 197
loading snake breed: 120
loading snake breed: 61
loading snake breed: 124
loading snake breed: 67
loading snake breed: 110
loading snake breed: 44
loading snake breed: 203
loading snake breed: 32
loading snake breed: 43
loading snake breed: 111
loading snake breed: 219
loading snake breed: 156
loading snake breed: 175
loading snake breed: 193
loading snake breed: 201
loading snake breed: 88
loading snake breed: 83
loading snake breed: 109
loading snake breed: 107
loading snake breed: 185
loading snake breed: 89
loading snake breed: 206
loading snake breed: 165
loading snake breed: 127
loading snake breed: 15
loading snake breed: 230
loading snake breed: 204
loading snake breed: 21
loading snake breed: 66
loading snake breed: 167
loading snake breed: 63
loading snake breed: 128
loading snake br

In [5]:
pairs, targets = loader.make_oneshot_task()

In [6]:
prediction = model.predict(pairs)

In [7]:
prediction

array([[0.4018709 ],
       [0.46314165],
       [0.3676334 ],
       [0.32479432],
       [0.46275142],
       [0.5228324 ],
       [0.4682466 ],
       [0.34014118],
       [0.32990766],
       [0.34315553],
       [0.40578827],
       [0.38399917],
       [0.429284  ],
       [0.34203216],
       [0.34073955],
       [0.38780743],
       [0.32813725],
       [0.29408592],
       [0.42393643],
       [0.4981438 ],
       [0.3961813 ],
       [0.36058804],
       [0.44851914],
       [0.41613993],
       [0.33638766],
       [0.5409753 ],
       [0.33673286],
       [0.44198704],
       [0.47398418],
       [0.51410407],
       [0.50079095],
       [0.5062505 ],
       [0.42209432],
       [0.47809243],
       [0.5469968 ],
       [0.4476906 ],
       [0.4734066 ],
       [0.49696046],
       [0.47375858],
       [0.5068689 ],
       [0.38793907],
       [0.51351655],
       [0.55401385],
       [0.4633619 ],
       [0.45737186],
       [0.4696424 ],
       [0.5106567 ],
       [0.483

In [8]:
targets

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [10]:
np.argmax(targets)

43

In [11]:
np.argmax(prediction)

62

In [12]:
prediction[43]

array([0.4633619], dtype=float32)

In [13]:
prediction[62]

array([0.5697352], dtype=float32)